<a href="https://colab.research.google.com/github/RitvikPrabhu/AutoCBC/blob/master/Milestone2_DiseaseDetect/RetinaNet/Disease_detect_RetinaNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**HemeAI: Disease Detect**

## The following notebook attempts to identify diseases using RetinaNet

###Author: Ritvik Prabhu

First let us import a tool that can clear extremely verbose outputs

In [1]:
from IPython.display import clear_output 


Below we will clone our repository. The repository will contain the model files, annotated dataset and setup.py which are all necessary to train the model used for the first milestone of the project.

In [2]:
!git clone https://github.com/RitvikPrabhu/AutoCBC.git
%cd AutoCBC
!apt-get install tk-dev python-tk
!pip install -r requirements.txt
%cd Milestone2_DiseaseDetect/RetinaNet
!git clone https://github.com/yhenon/pytorch-retinanet.git
!pip install pandas
!pip install pycocotools
!pip install opencv-python
!pip install requests
!pip install torch==1.13
!pip install torchvision==0.14.1
clear_output()


The code below will allows us to reset our directory without having to shutdown the entire runtime

In [3]:
# !rm -rf AutoCBC/

Obtain the Dataset (from Roboflow)

In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="k7t7R1s1qjS9eojgbD50")
project = rf.workspace().project("disease-detection-z1mqc")
dataset = project.version(3).download("retinanet")
clear_output()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=d6f384567152ea5f39676972d77d36034265e9be1a75decf0c06e8a688c5b3f7
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...

Delete empty lines in csv files

In [5]:
import csv

#For train
with open('Disease-Detection-3/train/_annotations.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    rows = [row for row in reader if any(field.strip() for field in row)]

with open('Disease-Detection-3/train/_annotations.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(rows)

#For valid
with open('Disease-Detection-3/valid/_annotations.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    rows = [row for row in reader if any(field.strip() for field in row)]

with open('Disease-Detection-3/valid/_annotations.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(rows)

Make sure that the coordinates in the annotations are rounded to the nearest integer

In [6]:
#For train data
with open('Disease-Detection-3/train/_annotations.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    rows = []
    for row in reader:
        # Convert columns 2-5 to integers
        row[1:5] = map(int, row[1:5])
        rows.append(row)

# Write back to the CSV file
with open('Disease-Detection-3/train/_annotations.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerows(rows)

#For validation data
with open('Disease-Detection-3/valid/_annotations.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    rows = []
    for row in reader:
        # Convert columns 2-5 to integers
        row[1:5] = map(int, row[1:5])
        rows.append(row)

# Write back to the CSV file
with open('Disease-Detection-3/valid/_annotations.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerows(rows)

Run the framework found here (https://github.com/yhenon/pytorch-retinanet)

In [ ]:
!mv Disease-Detection-3/train/*.jpg pytorch-retinanet/
!mv Disease-Detection-3/valid/*.jpg pytorch-retinanet/
%cd pytorch-retinanet/
!python train.py --dataset csv --csv_train ../Disease-Detection-3/train/_annotations.csv  --csv_classes ../class_list.csv  --csv_val ../Disease-Detection-3/valid/_annotations.csv